In [1]:
from windspharm.xarray import VectorWind
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt
import os 
import xesmf as xe

In [2]:
# ## +++ vars in: 
## +++ vars in: ERA5.h1.U200.1979010100000-1993123100000.nc

#veritical integral start and end: 
low_end = 1000 #mb
high_end = 760 #mb

###########################################################################
######################### Templates #######################################
###########################################################################

f1_in='f.e.FTORCHmjo_fullCNN_DT.cam.h1.U.1979010100000-1990123100000.nc'
var_u = 'U'
f2_in='f.e.FTORCHmjo_fullCNN_DT.cam.h1.V.1979010100000-1990123100000.nc'
var_v = 'V'
f3_in='f.e.FTORCHmjo_fullCNN_DT.cam.h1.Q.1979010100000-1990123100000.nc'
var_q = 'Q'
f_out='f.e.FTORCHmjo_fullCNN_DT.cam.h1.MCBOT.1979010100000-1990123100000.nc'
var_mc = 'MCBOT'
dir_in = '/glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_fullCNN_DT/ts/'

###########################################################################
######################### Templates #######################################
###########################################################################

# f1_in='f.e.FTORCHmjo_MEANmjo_DT.cam.h1.U.1979010100000-1990122200000.nc'
# var_u = 'U'
# f2_in='f.e.FTORCHmjo_MEANmjo_DT.cam.h1.V.1979010100000-1990122200000.nc'
# var_v = 'V'
# f3_in='f.e.FTORCHmjo_MEANmjo_DT.cam.h1.Q.1979010100000-1990122200000.nc'
# var_q = 'Q'
# f_out='f.e.FTORCHmjo_MEANmjo_DT.cam.h1.MCBOT.1979010100000-1990122200000.nc'
# var_mc = 'MCBOT'
# dir_in = '/glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_MEANmjo_DT/ts/'

###########################################################################
######################### Templates #######################################
###########################################################################

# f1_in='f.e.FTORCHmjo_CNTRLmjo_DT.cam.h1.U.1979010100000-1990122200000.nc'
# var_u = 'U'
# f2_in='f.e.FTORCHmjo_CNTRLmjo_DT.cam.h1.V.1979010100000-1990122200000.nc'
# var_v = 'V'
# f3_in='f.e.FTORCHmjo_CNTRLmjo_DT.cam.h1.Q.1979010100000-1990122200000.nc'
# var_q = 'Q'
# f_out='f.e.FTORCHmjo_CNTRLmjo_DT.cam.h1.MCBOT.1979010100000-1990122200000.nc'
# var_mc = 'MCBOT'
# dir_in = '/glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_CNTRLmjo_DT/ts/'
###########################################################################
######################### Templates #######################################
###########################################################################

# f1_in='f.e.FTORCHmjo_CNNmjo_DT.cam.h1.U.1979010100000-1990122200000.nc'
# var_u = 'U'
# f2_in='f.e.FTORCHmjo_CNNmjo_DT.cam.h1.V.1979010100000-1990122200000.nc'
# var_v = 'V'
# f3_in='f.e.FTORCHmjo_CNNmjo_DT.cam.h1.Q.1979010100000-1990122200000.nc'
# var_q = 'Q'
# f_out='f.e.FTORCHmjo_CNNmjo_DT.cam.h1.MCBOT.1979010100000-1990122200000.nc'
# var_mc = 'MCBOT'
# dir_in = '/glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_CNNmjo_DT/ts/'

## DASK

In [3]:
# ### ++++++ dask NCAR client: 
from distributed import Client
from dask_jobqueue import PBSCluster
project_num = 'P03010039' #what project key do you have? 

cluster = PBSCluster(account=project_num,walltime='12:00:00',cores=1, memory='40GB',shared_temp_directory='/glade/derecho/scratch/wchapman/tmp',queue='casper')
cluster.scale(jobs=40)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/diehard/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/diehard/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.110:43269,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/diehard/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
def regrid_small(inBIG,method='bilinear',level=None):
    """
    Regrid the input variable from a list of files to a predefined grid.

    Parameters:
    - file_list (list): List of input files.
    - varin (str): Input variable name.
    - varout (str): Output variable name.
    - method (str): Resampling method (default is 'bilinear').
    - level (str, optional): Vertical level to select (default is None).

    Returns:
    - xr.Dataset: Regridded dataset.
    """
        
    ds_out = xr.open_dataset('/glade/work/wchapman/ADF_obs_Mar2024/CERES_EBAF_Ed4.1_2001-2020.nc')
    fn ='./bilinear_192x288_180x360.nc'
    regridder = xe.Regridder(inBIG, ds_out, "bilinear", weights=fn)
    outSMALL = regridder(inBIG)
    return outSMALL

def regrid_big(inBIG,method='bilinear',level=None):
    """
    Regrid the input variable from a list of files to a predefined grid.

    Parameters:
    - file_list (list): List of input files.
    - varin (str): Input variable name.
    - varout (str): Output variable name.
    - method (str): Resampling method (default is 'bilinear').
    - level (str, optional): Vertical level to select (default is None).

    Returns:
    - xr.Dataset: Regridded dataset.
    """
        
    ds_out = xr.open_dataset('/glade/work/wchapman/miles-rollout/notebooks/gather_global_data/ML_1deg_grid.nc')
    regridder = xe.Regridder(inBIG, ds_out, "bilinear")
    outSMALL = regridder(inBIG)
    return outSMALL

In [5]:
%%time
file_path = os.path.join(dir_in, f1_in)  # Join the directory and file name
DS = xr.open_dataset(file_path).sel(lev=slice(high_end,low_end)).chunk({"time": 10,"lat":192,"lon":288,"lev":8})
# DS = DS.load()
print('1')
file_path = os.path.join(dir_in, f2_in)  # Join the directory and file name
DS2 = xr.open_dataset(file_path).sel(lev=slice(high_end,low_end)).chunk({"time": 10,"lat":192,"lon":288,"lev":8})
# DS2 = DS2.load()
print('2')
file_path = os.path.join(dir_in, f3_in)  # Join the directory and file name
DS3 = xr.open_dataset(file_path).sel(lev=slice(high_end,low_end)).chunk({"time": 10,"lat":192,"lon":288,"lev":8})
# DS3 = DS3.load()
print('3')

#regrid to small:
print('...regridding...')
DS2_small = regrid_small(DS2).drop('PS')
print('1')
DS_small = regrid_small(DS).drop('PS')
print('2')
DS3_small = regrid_small(DS3).drop('PS')
print('3')
print('... vector wind ...')


1
2
3
...regridding...
1
2
3
... vector wind ...
CPU times: user 2.59 s, sys: 148 ms, total: 2.74 s
Wall time: 3.48 s


## Load into Memory

In [6]:
%%time
DS2_small = DS2_small.compute()
print('1')
DS_small = DS_small.compute()
print('2')
DS3_small = DS3_small.compute()
print('3')

1
2
3
CPU times: user 17.2 s, sys: 1min 1s, total: 1min 18s
Wall time: 1min 46s


In [7]:
%%time
print('... Here we go ....')
# 
# We will have to do this by level ... 

print('....loading mcbot....')
DSmcbot = xr.zeros_like(DS['U']).to_dataset(name=var_mc).load()

for lvlv in DSmcbot['lev']:
    print('doing: ',lvlv.values)
    print('...vector wind....')
    w = VectorWind(DS_small[var_u].sel(lev=lvlv)*DS3_small[var_q].sel(lev=lvlv), DS2_small[var_v].sel(lev=lvlv)*DS3_small[var_q].sel(lev=lvlv))
    print('...w.vrtdiv()....')
    vrt, div = w.vrtdiv()
    div = div.to_dataset(name=var_mc)
    print('...regrid....')
    div = regrid_big(div.chunk({"time": 10,"lat": 180, "lon": 360}))
    print('.....loading....')
    div.load()
    print('...dataset....')
    DSmcbot[var_mc].sel(lev=lvlv)[:] = div[var_mc].values
    # break
DSmcbot[var_mc].attrs['long_name'] = 'Moisture Convergence s-1'
DSmcbot[var_mc].attrs['units'] = 's-1'  # Ensure the unit is correct for PRECT
# vp = vp.drop('VP200')
DSmcbot.to_netcdf(os.path.join(dir_in, f_out))
# div.to_netcdf(f_out)
print('vertical integral')
DSend = xr.open_dataset(file_path)
DSint = (1/9.816)*((DSmcbot*DSend['lev'].diff('lev').sel(lev=slice(high_end,low_end))).sum('lev'))
fsplit= f_out.split('cam.h1')
f_out2 = f'{fsplit[0]}cam.h1.VERTICALINT{fsplit[-1]}'
DSint.to_netcdf(os.path.join(dir_in, f_out2))

... Here we go ....
....loading mcbot....
doing:  763.404481112957
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  820.8583686500788
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  859.5347665250301
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  887.0202489197254
...vector wind....
...w.vrtdiv()....


IOStream.flush timed out

...regrid....


.....loading....
...dataset....
doing:  912.644546944648
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  936.1983984708786
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  957.485479535535
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  976.325407391414
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
doing:  992.556095123291
...vector wind....
...w.vrtdiv()....
...regrid....
.....loading....
...dataset....
vertical integral
CPU times: user 9min 9s, sys: 3min 14s, total: 12min 23s
Wall time: 12min 16s


In [8]:
print('...shutting down dask client...')
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

...shutting down dask client...
...shutdown client...


In [9]:
rm dask-worker*

In [10]:
DSint

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 4380)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1979-01-01 00:00:00 ... 1990-12-31 00:00:00
Data variables:
    MCBOT    (time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [8]:
ls -lathr /glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_meanGPU_exp001/ts/f.e.FTORCHmjo_meanGPU_exp001.cam.h1.Z500.1979010100000-2010123100000.nc

-rw-r--r-- 1 wchapman ncar 4.9G Apr 28 18:20 /glade/derecho/scratch/wchapman/ADF/f.e.FTORCHmjo_meanGPU_exp001/ts/f.e.FTORCHmjo_meanGPU_exp001.cam.h1.Z500.1979010100000-2010123100000.nc


In [15]:
import xarray as xr
xr.open_dataset('/glade/campaign/cgd/amp/wchapman/Reanalysis/ERA5_z500/ERA5.h1.Z500.camgrid.1979_2018.nc')

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 14610)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2018-12-31
Data variables:
    Z500     (time, lat, lon) float32 ...

In [23]:
DS = xr.open_dataset('/glade/campaign/cgd/amp/wchapman/Reanalysis/ERA5_obs/e5.moda.an.pl.128_131_z.ll025sc.camgrid.1979.2022.nc')